In [1]:
import os, time, re, json, subprocess, sys
import pandas as pd
import importlib.util as il

if None in [il.find_spec('python-ulid'), il.find_spec('pyperclip')]:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'python-ulid']);
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pyperclip']);
    
from ulid import ULID
import pyperclip

def gen_ulid():
    return str(ULID.from_timestamp(time.time()))

def convert_coord(c):
    c = str(c)
    j = len(c) - 6
    d = int(c[0:2 + j])
    m = int(c[2 + j:4 + j])
    s = float(c[4 + j:6 + j] + '.' + c[6 + j:])
    q = 1 if j == 0 else -1
    coord = round(q * (d + m / 60 + s / 3600), 6)
    
    return coord

def pprint(dict):
    print(json.dumps(dict, indent=2))

def comma_followed_by_number(s):
    for i, char in enumerate(s[:-1]):
        if char == ',' and s[i+1].isdigit():
            return True
    return False

def extract_table_section_from_file(section_header, filename, offset=0):
    offset *= 3
    section_header = '******* ' + section_header + ' *******'

    downloads_folder = os.path.join(os.path.expanduser("~"), "Downloads")
    with open(os.path.join(downloads_folder, filename), "r") as file:
        lines = file.readlines()

    extracted_lines = []
    inside_section = False
    end_marker_count = 0

    for line in lines:
        if section_header in line:
            inside_section = True
            extracted_lines.append(line)
            continue

        if inside_section:
            if end_marker_count > offset:
                extracted_lines.append(line)
            # Count lines that are mostly dashes
            if line.strip().startswith('---'):
                end_marker_count += 1
                if end_marker_count >= 3 + offset:
                    break

    return "".join(extracted_lines)

def remove_dash_lines(text):
    cleaned_lines = [
        line for line in text.splitlines()
        if not line.strip().startswith("---")
    ]
    return "\n".join(cleaned_lines)

def convert_pipe_text_to_csv(multi_line_text):
    csv_lines = []
    for line in multi_line_text.splitlines():
        if not line.strip():
            continue
        if '|' not in line:
            continue
        
        fields = [field.strip() for field in line.strip('|').split('|')]
        csv_line = '|'.join(fields)
        csv_lines.append(csv_line)

    return '\n'.join(csv_lines)

def csv_text_to_dataframe(csv_text):
    lines = [line.strip() for line in csv_text.strip().split('\n') if line.strip()]
    
    headers = [h.strip() for h in lines[0].split('|')]
    
    data = []
    for line in lines[1:]:
        fields = [f.strip() for f in line.split('|')]
        data.append(fields)
    
    df = pd.DataFrame(data, columns=headers)
    return df

def read_adaptation_section(section_header, filename, offset=0):
    text = extract_table_section_from_file(section_header, filename, offset)
    text = remove_dash_lines(text)
    text = convert_pipe_text_to_csv(text)
    
    return csv_text_to_dataframe(text)

In [2]:
downloads_path = os.path.join(os.path.expanduser("~"), "Downloads")
cols = ["Arts Id", "Route Type", "Route Id", "Owning Facility", "Route Fix",
    "ARTS Fix", "Unique Name", "Altitude Lower", "Altitude Upper", "Ac Class Criterias"]
df = pd.read_csv(os.path.join(downloads_path, "route_based.csv"), usecols=cols)

facility = df[df['Arts Id'] == 'TTT']

s = []
dp_df = facility[(facility['Route Type'] == 'DP') | (facility['Route Type'] == 'AIRWAY')]
for index, row in dp_df.iterrows():
    if row['Route Type'] == 'DP':
        route_id = re.sub(r'\d+$', '#', row['Route Id'])
    else:
        route_id = row['Route Id']
    data = route_id + (' ' + str(row['Route Fix'])).replace(' nan', '')
    pattern = row['ARTS Fix']
    
    if not pd.isna(row['Altitude Lower']):
        aoa = int(row['Altitude Lower'] / 100)
        data += ' AOA/' + f"{aoa:03d}"
    if not pd.isna(row['Altitude Upper']):
        aob = int(row['Altitude Upper'] / 100)
        data += ' AOB/' + f"{aob:03d}"
    if not pd.isna(row['Ac Class Criterias']):
        acc = row['Ac Class Criterias']
        if not('NATJ' in acc or 'NATM' in acc or \
               'ZMAQ' in acc or 'ZMAP' in acc):
            continue
        
        data += ' TYP/'
        if 'NATJ' in acc:
            data += 'J'
        if 'NATM' in acc:
            data += 'T'
        if 'ZMAQ' in acc or 'ZMAP' in acc:
            data += 'P'

    out = 'fix_pattern[\'' + data + '\'] = \'' + pattern + '\'\n'
    if not out in s:
        s.append(out)

adr_df = facility[facility['Route Type'] == 'ADR']
for index, row in adr_df.iterrows():
    if pd.isna(row['Route Fix']):
        continue
    elif row['Route Fix'][3:] == 'WX':
        continue
    
    data = row['Route Fix']
    if data in list(dp_df['Route Fix']):
        continue
    
    pattern = row['ARTS Fix']
    
    if not pd.isna(row['Altitude Lower']):
        aoa = int(row['Altitude Lower'] / 100)
        data += ' AOA/' + f"{aoa:03d}"
    if not pd.isna(row['Altitude Upper']):
        aob = int(row['Altitude Upper'] / 100)
        data += ' AOB/' + f"{aob:03d}"
    if not pd.isna(row['Ac Class Criterias']):
        acc = row['Ac Class Criterias']
        if not('NATJ' in acc or 'NATM' in acc or \
               'ZMAQ' in acc or 'ZMAP' in acc):
            continue
        
        data += ' TYP/'
        if 'NATJ' in acc:
            data += 'J'
        if 'NATM' in acc:
            data += 'T'
        if 'ZMAQ' in acc or 'ZMAP' in acc:
            data += 'P'

    out = 'fix_pattern[\'' + data + '\'] = \'' + pattern + '\'\n'
    if not out in s:
        s.append(out)

s_out = ''.join(sorted(s, key=lambda x: x.split("=")[1]))
pyperclip.copy(s_out)
print(s_out)

fix_pattern['LLRGO# WILON'] = 'BAM'
fix_pattern['BAYPO# WILON'] = 'BAR'
fix_pattern['PATOY'] = 'BAY'
fix_pattern['DORMR AOA/000 AOB/121'] = 'BLO'
fix_pattern['DORMR AOA/121 AOB/999'] = 'BLO'
fix_pattern['DORMR# SYKES'] = 'BLR'
fix_pattern['TIDES# GOLTY'] = 'BLR'
fix_pattern['V35 CHARO AOA/050 AOB/999'] = 'BOO'
fix_pattern['V579 VIOLA AOA/050 AOB/121'] = 'BOO'
fix_pattern['V97 TABIR AOA/000 AOB/110'] = 'DAR'
fix_pattern['DARBS'] = 'DAR'
fix_pattern['V97 PLYER AOA/081 AOB/999'] = 'END'
fix_pattern['MOMIE'] = 'END'
fix_pattern['V35 NESST AOA/000 AOB/060'] = 'ENE'
fix_pattern['V441 NITTS AOA/000 AOB/060'] = 'ENE'
fix_pattern['V97 PLYER AOA/000 AOB/060'] = 'ENE'
fix_pattern['V35 NESST AOA/060 AOB/999'] = 'ENF'
fix_pattern['V441 NITTS AOA/060 AOB/999'] = 'ENF'
fix_pattern['V97 PLYER AOA/060 AOB/081'] = 'ENF'
fix_pattern['LLRGO# LACEN'] = 'ENM'
fix_pattern['LLRGO# ENDED'] = 'ENM'
fix_pattern['ENDED# ENDED'] = 'ENR'
fix_pattern['TIDES# BRUNG'] = 'ENR'
fix_pattern['CEXAN AOA/000 AOB/101'] = 'FM

In [3]:
# TPA Scratchpads
fix_pattern = {}
fix_pattern['LLRGO# WILON'] = 'BAM'
fix_pattern['LLRGO# CAMJO'] = 'BAM'
fix_pattern['BAYPO#'] = 'BAR'
fix_pattern['PATOY'] = 'BAY'
fix_pattern['DORMR'] = 'BLO'
fix_pattern['DORMR#'] = 'BLR'
fix_pattern['TIDES# GOLTY'] = 'BLR'
fix_pattern['TIDES# FROOT'] = 'BLR'
fix_pattern['TIDES# CIGAR'] = 'BLR'

fix_pattern['CEXAN AOA/000 AOB/101'] = 'FME'

fix_pattern['CHARO AOA/050 AOB/999'] = 'BOO'
fix_pattern['VIOLA AOA/050 AOB/121'] = 'BOO'
fix_pattern['SABEE AOA/050 AOB/999'] = 'BOO'
fix_pattern['RSW AOA/050 AOB/999'] = 'BOO'
fix_pattern['MURDO AOA/050 AOB/999'] = 'BOO'
fix_pattern['V35 CHARO AOA/050 AOB/999'] = 'BOO'
fix_pattern['V579 VIOLA AOA/050 AOB/121'] = 'BOO'

fix_pattern['CHARO AOA/000 AOB/050'] = 'OGI'
fix_pattern['VIOLA AOA/000 AOB/050'] = 'OGI'
fix_pattern['SABEE AOA/000 AOB/050'] = 'OGI'
fix_pattern['RSW AOA/000 AOB/050'] = 'OGI'
fix_pattern['MURDO AOA/000 AOB/050'] = 'OGI'
fix_pattern['V35 CHARO AOA/000 AOB/050'] = 'OGI'
fix_pattern['V579 VIOLA AOA/000 AOB/050'] = 'OGI'

fix_pattern['LLRGO# LACEN'] = 'ENM'
fix_pattern['LLRGO# ENDED'] = 'ENM'
fix_pattern['ENDED#'] = 'ENR'
fix_pattern['TIDES# BRUNG'] = 'ENR'
fix_pattern['TIDES# CAMJO'] = 'ENR'
fix_pattern['TIDES# WILON'] = 'ENR'
fix_pattern['TIDES# MOMIE'] = 'ENR'

fix_pattern['V97 AOA/081 AOB/999'] = 'END'
fix_pattern['SZW AOA/081 AOB/999'] = 'END'
fix_pattern['MOMIE'] = 'END'
fix_pattern['ENDED'] = 'END'

fix_pattern['V97 AOA/000 AOB/060'] = 'ENE'
fix_pattern['SZW AOA/000 AOB/060'] = 'ENE'
fix_pattern['V35 NESST AOA/000 AOB/060'] = 'ENE'
fix_pattern['V35 CTY AOA/000 AOB/060'] = 'ENE'
fix_pattern['T495 AOA/000 AOB/060'] = 'ENE'
fix_pattern['T489 AOA/000 AOB/060'] = 'ENE'
fix_pattern['V441 AOA/000 AOB/060'] = 'ENE'

fix_pattern['V97 AOA/060 AOB/081'] = 'ENF'
fix_pattern['SZW AOA/060 AOB/081'] = 'ENF'
fix_pattern['V35 NESST AOA/060 AOB/999'] = 'ENF'
fix_pattern['V35 CTY AOA/060 AOB/999'] = 'ENF'
fix_pattern['T495 AOA/060 AOB/999'] = 'ENF'
fix_pattern['T489 AOA/060 AOB/999'] = 'ENF'
fix_pattern['V441 AOA/060 AOB/999'] = 'ENF'

fix_pattern['DARBS'] = 'DAR'

fix_pattern['V7 ROGAN'] = 'PHK'
fix_pattern['CROWD#'] = 'PKR'
fix_pattern['GANDY#'] = 'RSR'

fix_pattern['T341 WEZER AOA/065 AOB/121'] = 'GBS'
fix_pattern['T341 WEZER AOA/000 AOB/061'] = 'GIB'
fix_pattern['KNEED'] = 'KIZ'
fix_pattern['WOBAD AOA/000 AOB/130'] = 'KNR'
fix_pattern['WOBAD AOA/130 AOB/999'] = 'KNR'
fix_pattern['T210 PUNQU AOA/000 AOB/055'] = 'LAL'
fix_pattern['V152 JENSN AOA/000 AOB/055'] = 'LAL'
fix_pattern['V441 ODDEL AOA/000 AOB/055'] = 'LAL'
fix_pattern['V511 HALLR AOA/000 AOB/056'] = 'LAL'
fix_pattern['V533 CAMBE AOA/000 AOB/055'] = 'LAL'
fix_pattern['V581 LIMMO AOA/000 AOB/060'] = 'LAL'
fix_pattern['V7 NITTS AOA/000 AOB/060'] = 'LAL'
fix_pattern['YELLZ AOA/000 AOB/045'] = 'LAL'
fix_pattern['V581 LIMMO AOA/081 AOB/121'] = 'LAM'
fix_pattern['V7 NITTS AOA/081 AOB/999'] = 'LAM'
fix_pattern['T210 PUNQU AOA/056 AOB/121'] = 'LKE'
fix_pattern['V152 JENSN AOA/055 AOB/121'] = 'LKE'
fix_pattern['V441 ODDEL AOA/055 AOB/999'] = 'LKE'
fix_pattern['V533 CAMBE AOA/055 AOB/121'] = 'LKE'
fix_pattern['V581 LIMMO AOA/060 AOB/081'] = 'LKE'
fix_pattern['V7 NITTS AOA/060 AOB/081'] = 'LKE'
fix_pattern['YELLZ AOA/045 AOB/999'] = 'LKE'
fix_pattern['YOJIX'] = 'LKE'
fix_pattern['ORL'] = 'MCO'
fix_pattern['OCF'] = 'OCF'
fix_pattern['LAL'] = 'OCF'
fix_pattern['WEBBS'] = 'OCF'

    # 'A': ['LAL', 'YOJIX', 'V441', 'T336', 'T343'],
    # 'B': ['DORMR#', 'TIDES# CIGAR', 'TIDES# FROOT', 'DORMR', 'SYKES', 'CIGAR', 'FROOT'],
    # 'D': ['DARBS', 'V97 AOB 080', 'SZW'],
    # 'F': ['GANDY#', 'GANDY', 'PAIRS', 'MURDO', 'SABEE', 'SRQ', 'V579', 'RSW', 'CEXAN', 'CHARO', 'VIOLA'],
    # 'H': ['CROWD#', 'CROWD', 'HALLR', 'V509', 'FAZES'],
    # 'M': ['PRICY#', 'MINEE#', 'MINEE', 'PRICY'],
    # 'N': ['BAYPO#', 'BAYPO', 'CAMJO', 'WILON', 'NITTS AOA 100', 'V441', 'OCF', 'GNV'],
    # 'R': ['KNEED', 'V152', 'T210', 'ORL', 'VARZE'],
    # 'U': ['ENDED#', 'TIDES# MOMIE', 'TIDES# CAMJO', 'ENDED', 'LACEN', 'MOMIE', 'T495', 'T489', 'ATTAK', 'CTY'], 

In [202]:
def sort_fix_pattern(k):
    if '#' in k.split(' ')[0]:
        priority = 0
    elif any(c.isdigit() for c in k.split(' ')[0]):
        priority = 1
    else:
        priority = 2
    return (priority, k)

fix_pattern = dict(sorted(fix_pattern.items(), key=lambda item: sort_fix_pattern(item[0])))
# pprint(fix_pattern)

scratchpads = []
for s in fix_pattern:
    p = {}
    p['id'] = gen_ulid()

    sp = s
    if ' AOA/' in s:
        aoa = int(s.split(' AOA/')[1][0:3])
        if aoa != 0:
            p['minAltitude'] = aoa
        sp = re.sub(r' AOA/\d{3}', '', sp)
    if ' AOB/' in s:
        aob = int(s.split(' AOB/')[1][0:3])
        if aob <= 999:
            p['maxAltitude'] = aob
        sp = re.sub(r' AOB/\d{3}', '', sp)

    if ' DEP/' in s:
        p['airportIds'] = s.split(' DEP/')[1].split(' ')[0].split('/')
        sp = re.sub(r' DEP(/\w{1,})+', '', sp)
    
    p['searchPattern'] = sp
    p['template'] = fix_pattern[s]
    scratchpads.append(p)
pyperclip.copy(scratchpads)

In [188]:
# MIA Scratchpads
fix_pattern = {}
fix_pattern['AARPS'] = 'AAP'
fix_pattern['AGERS#'] = 'AGE'
fix_pattern['ALTNN'] = 'ALN'
fix_pattern['ALTNN#'] = 'ALT'
fix_pattern['BEECH TYP/P'] = 'BAQ'
fix_pattern['BEECH TYP/J'] = 'BEE'
fix_pattern['BNICE'] = 'BNC'
fix_pattern['BNGOS#'] = 'BNG'
fix_pattern['BNICE#'] = 'BNI'
fix_pattern['BNGOS'] = 'BNO'
fix_pattern['DORRL'] = 'DOL'
fix_pattern['AABER AOA/000 AOB/040'] = 'DOR'
fix_pattern['WUDIP AOA/000 AOB/040'] = 'DOR'
fix_pattern['COOFS AOA/000 AOB/040'] = 'DOR'
fix_pattern['DORM AOA/000 AOB/040'] = 'DOR'
fix_pattern['DORRL#'] = 'DRL'
fix_pattern['FEALX#'] = 'FEA'
fix_pattern['FEALX'] = 'FEL'
fix_pattern['FLMGO'] = 'FLG'
fix_pattern['FLMGO#'] = 'FLM'
fix_pattern['FOLZZ#'] = 'FOL'
fix_pattern['FOLZZ'] = 'FOZ'
fix_pattern['FRSBE'] = 'FRB'
fix_pattern['FRSBE#'] = 'FRS'
fix_pattern['GABOW#'] = 'GAB'
fix_pattern['GABOW'] = 'GAO'
fix_pattern['SANDL'] = 'GAO'
fix_pattern['GLADZ#'] = 'GLA'
fix_pattern['GWAVA#'] = 'GWA'
fix_pattern['GWAVA'] = 'GWV'
fix_pattern['HROCK'] = 'HRC'
fix_pattern['HROCK#'] = 'HRO'
fix_pattern['HURCN'] = 'HUC'
fix_pattern['HURCN#'] = 'HUR'
fix_pattern['HUSIL#'] = 'HUS'
fix_pattern['HUSIL'] = 'HUS'
fix_pattern['ELQUE'] = 'HUS'
fix_pattern['KLADA#'] = 'KLA'
fix_pattern['KLADA'] = 'KLA'
fix_pattern['KETLL'] = 'KTE'
fix_pattern['LIFRR# AOA/000 AOB/061'] = 'LI6'
fix_pattern['LIFRR# AOA/062 AOB/121'] = 'LI7'
fix_pattern['BOBOE AOA/000 AOB/061'] = 'LI6'
fix_pattern['BOBOE AOA/062 AOB/121'] = 'LI7'
fix_pattern['MAYNR'] = 'MAN'
fix_pattern['MAYNR#'] = 'MAY'
fix_pattern['MELLZ#'] = 'MEL'
fix_pattern['GLAMS'] = 'MEL'
fix_pattern['T208'] = 'PB4'
fix_pattern['T347'] = 'PB4'
fix_pattern['SHANC'] = 'PB4'
fix_pattern['VALKA'] = 'PB4'
fix_pattern['REGAE#'] = 'REG'
fix_pattern['SNAPR#'] = 'SNA'
fix_pattern['ZFP DEP/FLL/FXE/PMP'] = 'ZFX'
fix_pattern['ZFP DEP/MIA/TMB/OPF/HWO/HST/07FA'] = 'ZFZ'